In [1]:
from bokeh.layouts import column, row, gridplot
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, output_file, show
from bokeh.util.compiler import TypeScript
import numpy as np

In [2]:
a = [1,2,3,4,5,6,7,8,9]
a[::2]

[1, 3, 5, 7, 9]

In [56]:
# Define the exponential function
def exp_func(t, a):
    return np.exp(-a * t) * (t >= 0)

# Define the unit step function
def unit_func(t):
    return 1.0 * (t >= 0)

# Define the convolution function
def convolve(signal, impulse, time):
    dt = time[1] - time[0]
    result = np.zeros_like(time)
    for i in range(len(time)):
        # result[i] = np.sum(signal[:i+1] * impulse[i::-1]) * dt
        result[i] = np.sum(signal[:i+1] * impulse[i::-1][::-1]) * dt
    return result

def flip(signal, time, timeshift):
    dt = time[1] - time[0]
    shift = int((timeshift+2)/dt)
    signal_remove = signal[signal!=0]
    result = signal[shift::-1]
    result = np.pad(result, (0, 1000-len(result)), "constant")
    return result

def update_data(attrname, old, new):
    # Get the current value of the slider: current time
    a = slider.value
    pts = int(a*1000)
    # Update the values of the data source
    x_new = t[:pts]
    y_new = y[:pts]
    y2_source.data = dict(x=x_new, y=y_new)
    

In [81]:
# Generate input and impulse response signals
# Set up the time axis
t = np.linspace(-1, 5, 1000)
# Set up the functions to convolve
x = unit_func(t)
h = exp_func(t, 1)
hinv = flip(h, t, 4)
y = convolve(x, h, t)
# Set up Bokeh data sources for the signals
x_source = ColumnDataSource(data=dict(x=t, y1=x, y2=h))
h_source = ColumnDataSource(data=dict(x=t, y1=x, y2=hinv))
y_source = ColumnDataSource(data=dict(x=t, y=y))
y2_source = ColumnDataSource(data=dict(x=t[:1], y=y[:1]))
# Set up Bokeh figures for the signals
x_fig = figure(title='Signal', width=350, height=300, x_range=(-1, 4.01), y_range=(-1, 2))
# x_fig.varea(x='x', y1='y1', y2='y2', source=x_source, fill_alpha=0.5)
x_fig.line('x', 'y1', source=x_source, line_width=2, line_color='blue', legend_label="x(t)")
x_fig.line('x', 'y2', source=x_source, line_width=2, line_color='red', line_dash="dotted", legend_label="h(t)")
x_fig.legend.location = "top_left"
h_fig = figure(title='Shift', width=350, height=300, x_range=(-1, 4.01), y_range=(-1, 2))
h_fig.line('x', 'y1', source=h_source, line_width=2, line_color='blue', legend_label="x(t-t')")
h_fig.line('x', 'y2', source=h_source, line_width=2, line_color='red', line_dash="dotted", legend_label="h(t-t')")
h_fig.legend.location = "top_left"
y_fig = figure(title='Output Signal', width=700, height=200, x_range=(-1, 4.01), y_range=(-1, 2))
y_fig.line('x', 'y', source=y_source, line_width=2, line_color='green', line_alpha=0.2)
y_fig.line('x', 'y', source=y2_source, line_width=2, line_color='green', legend_label="x(t)*h(t)")
y_fig.legend.location = "top_left"

# shift_slider = Slider(title='t', value=-1, start=-1, end=5, step=0.006)
# shift_slider.on_change('value', update_data)

# Combine the figures and sliders into a Bokeh layout
# layout = row(x_fig, h_fig, y_fig)
layout = gridplot([[row(x_fig, h_fig)], [y_fig]])

# Display the Bokeh layout in a new browser window
output_file('convolution.html')
show(layout)

In [2]:
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import CustomJS, Select
import numpy as np

# Create the figure for the plot
plot = figure(width=600, height=400)

# Define the x-axis range for the plot
x = np.linspace(-5, 5, 100)

# Define the initial function to be displayed
y = np.zeros(len(x))

# Create a data source for the plot
source = {'x': x, 'y': y}
plot.line('x', 'y', source=source)

# Define the selector widget and its options
options = ['Unit Step', 'Dirac Impulse', 'Exponential']
selector = Select(title='Function:', value=options[0], options=options)

# Define the JavaScript callback for the selector
callback = CustomJS(args=dict(source=source, selector=selector), code="""
    const data = source.data;
    const func = selector.value;
    const x = data.x;
    let y = data.y;
    
    if (func === 'Unit Step') {
        y = x.map(val => val >= 0 ? 1 : 0);
    } else if (func === 'Dirac Impulse') {
        y = Array.from({length: x.length}, (_, i) => i === x.length / 2 ? 1 : 0);
    } else if (func === 'Exponential') {
        y = x.map(val => Math.exp(val));
    }
    
    source.change.emit();
""")

# Attach the callback to the selector
selector.js_on_change('value', callback)

# Create a layout with the plot and the selector
layout = column(plot, selector)

# Display the plot and the selector
show(layout)


In [ ]:
bokeh serve --show test.py